Collecting conflict related data from [ACLED](https://acleddata.com). To use this script, register with ACLED, obtain API keys and save the info as a json file in the following format:
```
{"ACLED":{
	"api_key":"YOUR KEY",
	"email":"YOUR EMAIL"
	}
}
```
See the [API GUIDE](https://apidocs.acleddata.com/) for more!

In [13]:
import requests
import json
import pandas as pd
import numpy as np
import os
import pycountry
from datetime import datetime, timedelta
from pandas import json_normalize

In [14]:
topfolder = '/Users/sahoko/GitHub/Food-insecurity-GP-forecasting/'
datafolder = os.path.join(topfolder,'Data/New/Conflict')
os.chdir(topfolder)

In [15]:
with open('Code/Python/api-keys.json') as f:
    key_info = json.load(f)

In [16]:
api_key = key_info['ACLED']['api_key']
email = key_info['ACLED']['email']
country = 'Nigeria'
st_date = (datetime.today() - timedelta(days=500)).strftime('%Y-%m-%d')

In [12]:
def ACLED_request(country, days_to_goback, num_day):
    today_dt = datetime.today()
    st_date_dt = today_dt - timedelta(days=days_to_goback)
    en_date_dt = st_date_dt + timedelta(days = num_day)
    st_date = st_date_dt.strftime('%Y-%m-%d')
    en_date = en_date_dt.strftime('%Y-%m-%d')
    r = requests.get(f"https://api.acleddata.com/acled/read?key={api_key}&email={email}&country={country}&event_date={st_date}|{en_date}&event_date_where=BETWEEN")
    print(r)
    r_tes = r.json()
    print([st_date,en_date])
    
    return json_normalize(r_tes['data'])

In [13]:
df = ACLED_request(country, 500,99)

<Response [200]>
['2022-10-14', '2023-01-21']


In [17]:
for i in [400,300,200,100]:
    df_tmp = ACLED_request(country, i, 99)
    print(len(df_tmp))
    df = pd.concat([df,df_tmp],axis=0)

<Response [200]>
['2023-01-22', '2023-05-01']
1275
<Response [200]>
['2023-05-02', '2023-08-09']
1170
<Response [200]>
['2023-08-10', '2023-11-17']
1379
<Response [200]>
['2023-11-18', '2024-02-25']
1223


In [30]:
df = df.sort_values(by=['admin1','event_date','event_id_cnty'])

In [84]:
df['fatalities']=pd.Series(df['fatalities'],dtype="Int64")
df['Datetime'] = pd.to_datetime(df['event_date'])
file = f"{datafolder}/{country}/ACLED-{country}-{st_date}-{df['event_date'].max()}.csv"
df.to_csv(file)

Creating data frame in a long format with the following columns:
- Date in datetime
- region: admin 1 name
- Number of conflicts 
- Fatalities

In [17]:
file = f"{datafolder}/{country}/ACLED-Nigeria-2022-10-14-2024-02-16.csv"
df = pd.read_csv(file, header = 0).iloc[:,1:]

In [18]:
region_list = list(df['admin1'].unique())
region_number = len(region_list)

In [19]:
date = list(pd.date_range(st_date, df['event_date'].max(),freq='D'))
date_length = len(date)

In [20]:
d = {'Datetime':list(pd.date_range(st_date, df['event_date'].max(),freq='D'))*region_number,
     'admin1':np.repeat(region_list, date_length)}
df1 = pd.DataFrame(data=d)
df1 = df1.sort_values(by=['admin1','Datetime'])

In [21]:
df2 = df.groupby(["Datetime", "admin1"])['fatalities'].agg(n_conflicts ='count', n_fatalities = 'sum').reset_index()
df2 = df2.sort_values(by=['admin1','Datetime'])
df2['Datetime'] = pd.to_datetime(df2['Datetime'])

In [22]:
df_merge = pd.merge(df1,df2,how="outer",on=['admin1','Datetime'],indicator=True)
df_merge = df_merge.fillna(0)

In [23]:
file = f"{datafolder}/{country}/ACLEDlong-{country}-{st_date}-{df['event_date'].max()}.csv"
df_merge.to_csv(file)

In [65]:
import plotly.express as px
fig = px.scatter(df_merge, x='n_conflicts', y='n_fatalities', color = 'admin1')
fig.show()